# Starbucks Capstone Challenge

### Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [1]:
import pandas as pd
import numpy as np
import math
import json
from datetime import datetime

from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [2]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [3]:
print("Portfolio Size = {}".format(portfolio.shape))
portfolio.head()

Portfolio Size = (10, 6)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [4]:
print("Profile Size = {}".format(profile.shape))
profile.head()

Profile Size = (17000, 5)


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
print("Transcript Size = {}".format(transcript.shape))
transcript.head()

Transcript Size = (306534, 4)


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
def check_nan(dataframe):
    nan_columns = dataframe.columns[dataframe.isna().any()].tolist()
    total = len(dataframe)
    print("Total : {}".format(total))
    for column in nan_columns:
        count = dataframe[column].isna().sum()
        print("{} : {}, remaining : {}".format(column, count, total - count))

In [7]:
check_nan(portfolio)

Total : 10


In [8]:
portfolio_nan = portfolio

In [9]:
check_nan(profile)

Total : 17000
gender : 2175, remaining : 14825
income : 2175, remaining : 14825


In [10]:
profile_nan = profile.dropna()
profile_nan = profile_nan.reset_index(drop=True)
profile_nan

,gender,age,id,became_member_on,income
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
2,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
3,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
4,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
14820,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
14821,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
14822,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
14823,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [11]:
check_nan(transcript)

Total : 306534


In [12]:
transcript_nan = transcript

In [13]:
def do_one_hot_encoding(dataframe, column_list):
    for column in column_list:
        column_dummies = pd.get_dummies(dataframe[column])
        dataframe = dataframe.drop(column, axis = 1)
        dataframe = dataframe.join(column_dummies)
    return dataframe

In [14]:
columns = ['offer_type']
portfolio_encoded = do_one_hot_encoding(portfolio_nan, columns)
portfolio_encoded.head()

,reward,channels,difficulty,duration,id,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0


In [15]:
columns = ['gender']
profile_encoded = do_one_hot_encoding(profile_nan, columns)
profile_encoded.head()

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0


In [16]:
columns = ['event']
transcript_encoded = do_one_hot_encoding(transcript_nan, columns)
transcript_encoded.head()

,person,value,time,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0


In [17]:
all_keys  = []
def display_all_keys(x):
   
    if x.keys() not in all_keys:
        all_keys.append(x.keys())

In [18]:
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [19]:
def change_name(x, from_name, to_name):
    if from_name in x.keys():
        x[to_name] = x.pop(from_name)
    return x

In [20]:
transcript_encoded['value'].apply(lambda x: change_name(x, 'offer id', 'offer_id'))

0         {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer_id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [21]:
all_keys = []
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer_id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [22]:
def check_not_offer(x, column):
    if column in x.keys() :
        return x[column]

In [23]:
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, 'offer_id'))

In [24]:
offer_ids = offer_ids.to_frame()
offer_ids.columns = ['offer_id']
offer_ids.head()

,offer_id
0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,2906b810c7d4411798c6938adc9daaa5
3,fafdcd668e3743c1bb461111dcafc2a4
4,4d5c57ea9a6940dd891ad53e9dbe8da0


In [25]:
transcript_offer_id = transcript_encoded.join(offer_ids)

In [26]:
transcript_offer_id

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,{'amount': 1.5899999999999999},714,0,0,0,1,None
306530,68213b08d99a4ae1b0dcb72aebd9aa35,{'amount': 9.53},714,0,0,0,1,None
306531,a00058cf10334a308c68e7631c529907,{'amount': 3.61},714,0,0,0,1,None
306532,76ddbd6576844afe811f1a3c0fbb5bec,{'amount': 3.5300000000000002},714,0,0,0,1,None


In [27]:
column = 'amount'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids

,amount
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
306529,1.59
306530,9.53
306531,3.61
306532,3.53


In [28]:
transcript_amount = transcript_offer_id.join(offer_ids)
transcript_amount.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [29]:
column = 'reward'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids.dropna()

,reward
12658,2.0
12672,2.0
12679,5.0
12692,10.0
12697,10.0
...,...
306475,3.0
306497,3.0
306506,2.0
306509,2.0


In [30]:
transcript_reward = transcript_amount.join(offer_ids)
transcript_reward.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [31]:
transcript_on_value = transcript_reward.drop('value', axis = 1)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,NaN


In [32]:
portfolio_encoded

,reward,channels,difficulty,duration,id,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0
5,3,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0
6,2,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0
7,0,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1
8,5,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0
9,2,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0


In [33]:
def list_to_dataframe(x, column):
    if column in x:
        return 1
    else:
        return 0

In [34]:
values = ['email', 'web', 'mobile', 'social']
for i in values:
    portfolio_encoded_series = portfolio_encoded['channels'].apply(lambda x: list_to_dataframe(x, i))
    portfolio_encoded_series = portfolio_encoded_series.to_frame()
    portfolio_encoded_series.columns = [i]
    portfolio_encoded = portfolio_encoded.join(portfolio_encoded_series)
portfolio_encoded

,reward,channels,difficulty,duration,id,bogo,discount,informational,email,web,mobile,social
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,1,0
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,1,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,1,0,0
5,3,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,0,1,1
8,5,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,1,0


In [35]:
portfolio_channels = portfolio_encoded.drop('channels', axis = 1)
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,email,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,1,0


In [36]:
#if 'email' is same in every row then its redundant
portfolio_channels = portfolio_channels.drop('email', axis = 1)
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [37]:
transcript_on_value['reward'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,0.0


In [38]:
transcript_on_value['amount'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,0.0


In [39]:
p_columns = portfolio_channels.columns
p_columns = p_columns.to_list()

In [40]:
for n, i in enumerate(p_columns):
    if i!='offer_id':
        p_columns[n] = 'offer_' + p_columns[n]
p_columns

['offer_reward',
 'offer_difficulty',
 'offer_duration',
 'offer_id',
 'offer_bogo',
 'offer_discount',
 'offer_informational',
 'offer_web',
 'offer_mobile',
 'offer_social']

In [41]:
portfolio_channels.columns = p_columns
portfolio_channels

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [42]:
def convert_to_datetime(x):
    value = datetime.strptime(str(x), '%Y%m%d')
    return value

In [43]:
profile_encoded['became_member_on'] = profile_encoded['became_member_on'].apply(lambda x: convert_to_datetime(x))

In [44]:
profile_became_member_on = profile_encoded
profile_became_member_on

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1,0
...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,1,0,0
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,0,1,0
14822,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,1,0,0


In [45]:
profile_became_member_on['became_member_on_year'] = profile_became_member_on['became_member_on'].apply(lambda x: x.year)
profile_became_member_on

,age,id,became_member_on,income,F,M,O,became_member_on_year
0,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0,2017
1,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0,2017
2,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1,0,2018
3,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1,0,2018
4,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1,0,2017
...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,1,0,0,2018
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,0,1,0,2018
14822,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,0,1,0,2017
14823,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,1,0,0,2016


In [46]:
profile_became_member_on['became_member_on_month'] = profile_became_member_on['became_member_on'].apply(lambda x: x.month)
profile_became_member_on

,age,id,became_member_on,income,F,M,O,became_member_on_year,became_member_on_month
0,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0,2017,7
1,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0,2017,5
2,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1,0,2018,4
3,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1,0,2018,2
4,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1,0,2017,11
...,...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,1,0,0,2018,6
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,0,1,0,2018,7
14822,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,0,1,0,2017,1
14823,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,1,0,0,2016,3


In [47]:
profile_became_member_on = profile_became_member_on.drop('became_member_on', axis = 1)
profile_became_member_on

,age,id,income,F,M,O,became_member_on_year,became_member_on_month
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,2017,7
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,2017,5
2,68,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,2018,4
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2018,2
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,2017,11
...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,1,0,0,2018,6
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,0,1,0,2018,7
14822,49,01d26f638c274aa0b965d24cefe3183f,73000.0,0,1,0,2017,1
14823,83,9dc1421481194dcd9400aec7c9ae6366,50000.0,1,0,0,2016,3


In [48]:
column = 'became_member_on_year'
column_dummies = pd.get_dummies(profile_became_member_on[column])
profile_became_member_on = profile_became_member_on.drop(column, axis = 1)
profile_became_member_on = profile_became_member_on.join(column_dummies)
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,7,0,0,0,0,1,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,5,0,0,0,0,1,0
2,68,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,4,0,0,0,0,0,1
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2,0,0,0,0,0,1
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,11,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,1,0,0,6,0,0,0,0,0,1
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,0,1,0,7,0,0,0,0,0,1
14822,49,01d26f638c274aa0b965d24cefe3183f,73000.0,0,1,0,1,0,0,0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,50000.0,1,0,0,3,0,0,0,1,0,0


In [49]:
profile_became_member_on['income'].describe()

count     14825.000000
mean      65404.991568
std       21598.299410
min       30000.000000
25%       49000.000000
50%       64000.000000
75%       80000.000000
max      120000.000000
Name: income, dtype: float64

In [50]:
profile_became_member_on['income'] = profile_became_member_on['income'] / 120000.000000
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,55,0610b486422d4921ae7d2bf64640c50b,0.933333,1,0,0,7,0,0,0,0,1,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0
2,68,e2127556f4f64592b11af22de27a7932,0.583333,0,1,0,4,0,0,0,0,0,1
3,65,389bc3fa690240e798340f5a15918d5c,0.441667,0,1,0,2,0,0,0,0,0,1
4,58,2eeac8d8feae4a8cad5a6af0499a211d,0.425000,0,1,0,11,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,0.450000,1,0,0,6,0,0,0,0,0,1
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,0.600000,0,1,0,7,0,0,0,0,0,1
14822,49,01d26f638c274aa0b965d24cefe3183f,0.608333,0,1,0,1,0,0,0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,0.416667,1,0,0,3,0,0,0,1,0,0


In [51]:
profile_became_member_on['age'].describe()

count    14825.000000
mean        54.393524
std         17.383705
min         18.000000
25%         42.000000
50%         55.000000
75%         66.000000
max        101.000000
Name: age, dtype: float64

In [52]:
profile_became_member_on['age'] = profile_became_member_on['age'] / 101
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,0.544554,0610b486422d4921ae7d2bf64640c50b,0.933333,1,0,0,7,0,0,0,0,1,0
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0
2,0.673267,e2127556f4f64592b11af22de27a7932,0.583333,0,1,0,4,0,0,0,0,0,1
3,0.643564,389bc3fa690240e798340f5a15918d5c,0.441667,0,1,0,2,0,0,0,0,0,1
4,0.574257,2eeac8d8feae4a8cad5a6af0499a211d,0.425000,0,1,0,11,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,0.445545,6d5f3a774f3d4714ab0c092238f3a1d7,0.450000,1,0,0,6,0,0,0,0,0,1
14821,0.603960,2cb4f97358b841b9a9773a7aa05a9d77,0.600000,0,1,0,7,0,0,0,0,0,1
14822,0.485149,01d26f638c274aa0b965d24cefe3183f,0.608333,0,1,0,1,0,0,0,0,1,0
14823,0.821782,9dc1421481194dcd9400aec7c9ae6366,0.416667,1,0,0,3,0,0,0,1,0,0


In [53]:
transcript_profile = pd.merge(transcript_on_value, profile_became_member_on, left_on = ['person'], right_on = ['id'])
transcript_profile

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,0,0,0,1,None,19.89,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
3,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,5.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
4,78afa995795e4d85b5d9ceeca43f5fef,144,0,0,0,1,None,17.78,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272757,9fcbff4f8d7241faa4ab8a9d19c8a812,504,0,0,1,0,3f207df678b143eea3cee63160fa8bed,0.00,0.0,0.465347,...,0,1,0,10,0,0,0,0,1,0
272758,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0,0.465347,...,0,1,0,10,0,0,0,0,1,0
272759,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0,0.465347,...,0,1,0,10,0,0,0,0,1,0
272760,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0,0.574257,...,1,0,0,10,0,0,0,1,0,0


In [54]:
profile_became_member_on[profile_became_member_on['id'] == '78afa995795e4d85b5d9ceeca43f5fef']

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0


In [55]:
transcript_profile.shape

(272762, 22)

In [56]:
transcript_profile_portfolio = pd.merge(transcript_profile, portfolio_channels, left_on = ['offer_id'], right_on = ['offer_id'])
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.475248,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.574257,...,0,10,10,5,1,0,0,1,1,1


In [57]:
transcript_profile_portfolio.to_csv("transcript_profile_portfolio_test.csv", index=False)
transcript_profile_portfolio = pd.read_csv("transcript_profile_portfolio_test.csv")
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.475248,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.574257,...,0,10,10,5,1,0,0,1,1,1


In [58]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer_id', axis=1)
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,id,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,0.0,0.0,0.475248,8578196a074a4f328976e334fa9383a3,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,0.0,0.0,0.465347,9fcbff4f8d7241faa4ab8a9d19c8a812,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,0.0,0.0,0.465347,9fcbff4f8d7241faa4ab8a9d19c8a812,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,0.0,0.0,0.574257,3045af4e98794a04a5542d3eac939b1f,...,0,10,10,5,1,0,0,1,1,1


In [59]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('id', axis=1)
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,income,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,0.583333,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,0.583333,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,0.0,0.0,0.475248,0.483333,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,0.0,0.0,0.465347,0.783333,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,0.0,0.0,0.465347,0.783333,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,0.0,0.0,0.574257,0.650000,...,0,10,10,5,1,0,0,1,1,1


In [60]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('person', axis = 1)

In [61]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer received', axis = 1)

In [62]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer viewed', axis = 1)
transcript_profile_portfolio

,time,offer completed,transaction,amount,reward,age,income,F,M,O,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,...,0,5,5,7,1,0,0,1,1,0
1,6,0,0,0.0,0.0,0.742574,0.833333,1,0,0,...,0,5,5,7,1,0,0,1,1,0
2,132,1,0,0.0,5.0,0.742574,0.833333,1,0,0,...,0,5,5,7,1,0,0,1,1,0
3,408,0,0,0.0,0.0,0.673267,0.583333,0,1,0,...,1,5,5,7,1,0,0,1,1,0
4,420,0,0,0.0,0.0,0.673267,0.583333,0,1,0,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,504,0,0,0.0,0.0,0.475248,0.483333,0,1,0,...,1,10,10,5,1,0,0,1,1,1
148801,576,0,0,0.0,0.0,0.465347,0.783333,0,1,0,...,0,10,10,5,1,0,0,1,1,1
148802,576,0,0,0.0,0.0,0.465347,0.783333,0,1,0,...,0,10,10,5,1,0,0,1,1,1
148803,576,0,0,0.0,0.0,0.574257,0.650000,1,0,0,...,0,10,10,5,1,0,0,1,1,1


In [63]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('time', axis = 1)
transcript_profile_portfolio

,offer completed,transaction,amount,reward,age,income,F,M,O,became_member_on_month,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,5,7,1,0,0,1,1,0
1,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,5,7,1,0,0,1,1,0
2,1,0,0.0,5.0,0.742574,0.833333,1,0,0,5,...,0,5,5,7,1,0,0,1,1,0
3,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,5,7,1,0,0,1,1,0
4,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,0,0,0.0,0.0,0.475248,0.483333,0,1,0,6,...,1,10,10,5,1,0,0,1,1,1
148801,0,0,0.0,0.0,0.465347,0.783333,0,1,0,10,...,0,10,10,5,1,0,0,1,1,1
148802,0,0,0.0,0.0,0.465347,0.783333,0,1,0,10,...,0,10,10,5,1,0,0,1,1,1
148803,0,0,0.0,0.0,0.574257,0.650000,1,0,0,10,...,0,10,10,5,1,0,0,1,1,1


In [64]:
transcript_profile_portfolio.shape

(148805, 25)

In [65]:
column_name = 'offer completed'
cols = list(transcript_profile_portfolio.columns.values)
cols.pop(cols.index(column_name))
cols.append(column_name)
transcript_profile_portfolio.columns = cols
transcript_profile_portfolio

,transaction,amount,reward,age,income,F,M,O,became_member_on_month,2013,...,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social,offer completed
0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,5,7,1,0,0,1,1,0
1,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,5,7,1,0,0,1,1,0
2,1,0,0.0,5.0,0.742574,0.833333,1,0,0,5,...,0,5,5,7,1,0,0,1,1,0
3,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,5,7,1,0,0,1,1,0
4,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,0,0,0.0,0.0,0.475248,0.483333,0,1,0,6,...,1,10,10,5,1,0,0,1,1,1
148801,0,0,0.0,0.0,0.465347,0.783333,0,1,0,10,...,0,10,10,5,1,0,0,1,1,1
148802,0,0,0.0,0.0,0.465347,0.783333,0,1,0,10,...,0,10,10,5,1,0,0,1,1,1
148803,0,0,0.0,0.0,0.574257,0.650000,1,0,0,10,...,0,10,10,5,1,0,0,1,1,1


In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNet, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.fc2(out)
#         print("before = ", out)
        out = self.sig(out)
#         print("after = ", out)
        return out

In [67]:
def split_train_test(dataframe_values, train_percent = 0.7):
    test_size = int(dataframe_values.shape[0] * train_percent)
    np.random.shuffle(dataframe_values)
    train_data = dataframe_values[:test_size]
    test_data = dataframe_values[test_size:]
    print("train size = ", train_data.shape)
    print("test size = ", test_data.shape)
    print("total size = ", test_data.shape[0] + train_data.shape[0])
    return train_data, test_data

In [68]:
train_data, test_data = split_train_test(transcript_profile_portfolio.values)

train size =  (104163, 25)
test size =  (44642, 25)
total size =  148805


In [69]:
def get_loaders(data, batch_size = 64):
    x = torch.from_numpy(data[:, :-1]).float().squeeze()
    y = torch.from_numpy(data[:, -1]).float()
    
    ds = torch.utils.data.TensorDataset(x, y)
    loader = torch.utils.data.DataLoader(ds, batch_size = batch_size)
    return loader

In [70]:
train_loader = get_loaders(train_data)
train_loader

In [71]:
test_loader = get_loaders(test_data)
test_loader

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_model = SimpleNet(24, 10, 1).to(device)
binary_model

SimpleNet(
  (fc1): Linear(in_features=24, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (sig): Sigmoid()
)

In [73]:
import torch.optim as optim

optimizer = optim.SGD(binary_model.parameters(), lr = 0.001, momentum=0.5)
criterion = nn.BCELoss()

In [74]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    
    for epoch in range(0, epochs):
        model.train()
        
        total_loss = 0
        
        for batch in train_loader:
            
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(batch_x)
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
            
        print("Epoch: {}, Loss: {}".format(epoch, total_loss/ len(train_loader)))
        
    return model

In [75]:
trained_model = train(binary_model, train_loader, 10, criterion, optimizer, device)

D:\Anaconda\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([35])) that is different to the input size (torch.Size([35, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0, Loss: 0.5728605658328504
Epoch: 1, Loss: 0.4682536087838672
Epoch: 2, Loss: 0.4102818472196488
Epoch: 3, Loss: 0.3763451661722373
Epoch: 4, Loss: 0.353341880407761
Epoch: 5, Loss: 0.3351448595670283
Epoch: 6, Loss: 0.31909227341926066
Epoch: 7, Loss: 0.30280714886633714
Epoch: 8, Loss: 0.2892847620499954
Epoch: 9, Loss: 0.27473754005584433


In [150]:
def test(model, test_loader, criterion, device):
    with torch.no_grad():
        true_labels = []
        predicted_labels = []
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        for data, target in test_loader:
            try:
                data, target = data.to(device), target.to(device)
                output = model(data)
            
                output_label = output.cpu().detach().numpy()
#                 print(output_label.shape)
                predicted_labels.extend(output_label.squeeze())
#                 print("okoknot")
                true_labels.extend(target.cpu().detach().numpy().squeeze())
                        
#                 print("okok")
#             test_loss += criterion(output, target).data.item()
            
                for i in range(0, len(output_label)):
                    if(output[i].round()==target[i]):
                        correct += 1
                    total += 1
            except:
                None
            
        print("accuracy = {}".format(correct/total))
        return np.asarray(true_labels), np.asarray(predicted_labels)

In [77]:
true_labels, predicted_labels = test(trained_model, test_loader, criterion, device)
print(len(true_labels))
print(len(predicted_labels))

D:\Anaconda\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([34])) that is different to the input size (torch.Size([34, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


accuracy = 0.8359392500336007
44642
44642


In [78]:
f1_score(true_labels, predicted_labels.round())

0.8891109496124031

### KNeighborsClassifier

In [79]:
kclassifier = KNeighborsClassifier()

In [80]:
kclassifier.fit(train_data[:, :-1], train_data[:, -1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [81]:
kclassifier.score(test_data[:, :-1], test_data[:, -1])

1.0

In [82]:
kclassifier_predicted = kclassifier.predict(test_data[:, :-1])

In [83]:
f1_score(test_data[:, -1], kclassifier_predicted.round())

1.0

### GaussianNB

In [84]:
gaussian = GaussianNB()

In [85]:
gaussian.fit(train_data[:, :-1], train_data[:, -1])

GaussianNB(priors=None, var_smoothing=1e-09)

In [86]:
gaussian.score(test_data[:, :-1], test_data[:, -1])

0.6793826441467676

In [87]:
gaussian_predicted = gaussian.predict(test_data[:, :-1])

In [88]:
f1_score(test_data[:, -1], gaussian_predicted.round())

0.7297649391107335

### Validating the model

In [89]:
df = transcript_profile_portfolio.iloc[:, :-1]
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,transaction,amount,reward,age,income,F,M,O,became_member_on_month,2013,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,1,0,5,5,7,1,0,0,1,1
1,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,1,0,5,5,7,1,0,0,1,1
2,1,0,0.0,5.0,0.742574,0.833333,1,0,0,5,...,1,0,5,5,7,1,0,0,1,1
3,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,0,1,5,5,7,1,0,0,1,1
4,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,0,1,5,5,7,1,0,0,1,1


In [90]:
y = transcript_profile_portfolio.iloc[:, 24:]
print(type(y))
y.head()

<class 'pandas.core.frame.DataFrame'>


,offer completed
0,0
1,0
2,0
3,0
4,0


In [91]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True)

In [151]:
binary_model_valid = SimpleNet(24, 10, 1).to(device)
optimizer = optim.SGD(binary_model_valid.parameters(), lr = 0.001, momentum=0.5)
criterion = nn.BCELoss()

In [152]:
def get_loaders_validating(x, y, batch_size = 64):
    print(x.shape)
    print(y.shape)
    print("---")
    X = torch.from_numpy(x).float().squeeze()
    Y = torch.from_numpy(y).float()
    ds = torch.utils.data.TensorDataset(X, Y)
    loader = torch.utils.data.DataLoader(ds, batch_size = batch_size)
    return loader

In [153]:
scores = []
for i in range(5):
    result = next(kf.split(transcript_profile_portfolio), None)
    x_train = df.iloc[result[0]]
    x_test = df.iloc[result[1]]
    y_train = y.iloc[result[0]]
    y_test = y.iloc[result[1]]
    
#     print(x_train[:])
    
    
    train_loader = get_loaders_validating(x_train.values, y_train.values)
    test_loader = get_loaders_validating(x_test.values, y_test.values)
    
    model = train(binary_model_valid, train_loader, 10, criterion, optimizer, device)
    true_labels, predicted_labels = test(model, test_loader, criterion, device)
    scores.append(f1_score(true_labels, predicted_labels.round()))
    
print("Scores from each Iteration: ", scores)
print("Average K-Fold Score: ", np.mean(scores))

(119044, 24)
(119044, 1)
---
(29761, 24)
(29761, 1)
---
Epoch: 0, Loss: 0.3719683598590868
Epoch: 1, Loss: 0.44796066923984784
Epoch: 2, Loss: 0.3984947868060843
Epoch: 3, Loss: 0.3801473298183633
Epoch: 4, Loss: 0.37453532132706946
Epoch: 5, Loss: 0.35927369357084216
Epoch: 6, Loss: 0.3365022141360295
Epoch: 7, Loss: 0.3337743147300113
Epoch: 8, Loss: 0.3128861711712513
Epoch: 9, Loss: 0.3091765496721512
accuracy = 0.7139784946236559
(119044, 24)
(119044, 1)
---
(29761, 24)
(29761, 1)
---
Epoch: 0, Loss: 0.29800975588845935
Epoch: 1, Loss: 0.30927061081169516
Epoch: 2, Loss: 0.2903948960867588
Epoch: 3, Loss: 0.27925559451212667
Epoch: 4, Loss: 0.2680212299613215
Epoch: 5, Loss: 0.257782343116507
Epoch: 6, Loss: 0.2505755204560567
Epoch: 7, Loss: 0.24296664550263583
Epoch: 8, Loss: 0.23475441622564008
Epoch: 9, Loss: 0.22839959669099547
accuracy = 0.8449596774193548
(119044, 24)
(119044, 1)
---
(29761, 24)
(29761, 1)
---
Epoch: 0, Loss: 0.2257731334937138
Epoch: 1, Loss: 0.21819550995